In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from GNN import TextGCN, GCN
from utils import load_corpus

In [6]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus("R8")

FileNotFoundError: [Errno 2] No such file or directory: 'data/ind.R8.x'